In [13]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType

import reverse_geocoder as rg

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 17, Finished, Available, Finished)

In [14]:
df = spark.read.table('tb_earthquake_events_silver')

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 18, Finished, Available, Finished)

In [15]:
display(df)

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 22aa3892-5215-439c-bd89-df7949b62965)

In [16]:
coordinates = (11.0588, -86.9377)
rg.search(coordinates)[0].get('cc')

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 20, Finished, Available, Finished)

'NI'

In [17]:
def get_country_code(lat, lon):
    """
    Retrive the country code for a given latitude and longitude.

    Parameter:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Example: 
    >>> get_country_code(48.858843, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 21, Finished, Available, Finished)

In [18]:
# registring the udfs so they used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 22, Finished, Available, Finished)

In [19]:
# adding country code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitiude"), col("longitude")))

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 23, Finished, Available, Finished)

In [20]:
display(df_with_location)

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 994d5041-9bae-42a5-a605-b38ab5168911)

In [26]:
# Adding significance classification
df_with_location_sig_class = df_with_location.\
    withColumn(
        'sig_class',
        when(col("sig") < 100, "Low").
        when((col("sig") >= 100) & (col("sig") < 500), "Moderate").
        otherwise("High")
    )


StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 30, Finished, Available, Finished)

In [27]:
display(df_with_location_sig_class)

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c0f0a7f8-cb23-4688-ab5d-cad889c562d6)

In [29]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('tb_earthquake_events_gold')

StatementMeta(, 392f3cbd-70d0-4149-8453-943310e74d0c, 33, Finished, Available, Finished)